In [1]:
import pandas as pd
import numpy as np
import sys
import torch
import torch.nn as nn

In [2]:
df_train = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_train.csv")
df_test = pd.read_csv(r"C:\Users\maxim\Desktop\js\linfit\test\archive\mnist_test.csv")

In [3]:
xcolnams = df_train.columns.tolist()
print("check0")
df_train[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

Xs_train = torch.tensor(df_train[xcolnams[1:]].values).float().cuda()
print("check2")

ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_train[coln] = (df_train["label"] == i).astype(float)

check0
check1
check2


In [4]:
xcolnams = df_test.columns.tolist()
print("check0")
df_test[xcolnams[1:]] /= 255. #convert the pixel values to 0..1
print("check1")

Xs_test = torch.tensor(df_test[xcolnams[1:]].values).float().cuda()
print("check2")

ycolnams = []
for i in range(10):
    coln = "is"+str(i)
    ycolnams.append(coln)
    df_test[coln] = (df_test["label"] == i).astype(float)

check0
check1
check2


In [5]:
Ys_train = torch.tensor(df_train[ycolnams].values).float().cuda()
Ys_test = torch.tensor(df_test[ycolnams].values).float().cuda()

In [6]:
trainloader=torch.utils.data.DataLoader(list(zip(Xs_train, torch.tensor(df_train["label"].values).cuda())), batch_size=32, shuffle=True)
testloader=torch.utils.data.DataLoader(list(zip(Xs_test, torch.tensor(df_test["label"].values).cuda())), batch_size=32, shuffle=False)

In [7]:
print(Xs_test.shape)
print(Ys_train[:,:10])

torch.Size([10000, 784])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.]], device='cuda:0')


In [8]:
mod = nn.Sequential(
    nn.Linear(28**2, 400),
    nn.ReLU(),
    nn.Linear(400, 10),
    nn.ReLU()
)

# mod = nn.Sequential(
#     nn.Linear(28**2, 10),
# )

mod = mod.cuda()

In [9]:
Xs_train.shape

torch.Size([60000, 784])

In [10]:
mod(Xs_train)

tensor([[0.0043, 0.0148, 0.0615,  ..., 0.0000, 0.0000, 0.0000],
        [0.0225, 0.1318, 0.0661,  ..., 0.0000, 0.0000, 0.0000],
        [0.0184, 0.1373, 0.0640,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0014, 0.0347, 0.0454,  ..., 0.0000, 0.0000, 0.0000],
        [0.0059, 0.1062, 0.0176,  ..., 0.0000, 0.0000, 0.0000],
        [0.0093, 0.0798, 0.0743,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [11]:
import linstorch

In [12]:
import importlib
importlib.reload(linstorch)

<module 'linstorch' from 'c:\\Users\\maxim\\Desktop\\js\\linfit\\fulltorch\\linstorch.py'>

In [13]:
import time

In [14]:
forw = mod.forward(Xs_test)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_test["label"].values) / len(guessed))

0.0719


In [15]:
# t0 = time.time()
# mod = linstorch.solvemodel(mod, Xs_train, Ys_train)
# print('{} seconds'.format(time.time() - t0))

2.2500882148742676 seconds


In [16]:
forw = mod.forward(Xs_train)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_train["label"].values) / len(guessed))

0.90695


In [17]:
forw = mod.forward(Xs_test)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_test["label"].values) / len(guessed))

0.901


In [18]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(mod.parameters())
#optimizer = optim.SGD(mod.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adagrad(mod.parameters()) #optim.Adagrad(mod.parameters(), lr=0.001)

def train(ep):
    for epoch in range(ep):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # print(labels.shape)
            #labels = torch.argmax(labels, axis=1)
            # print(labels)
            # print(labels.shape)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = mod(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                    (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

    print('Finished Training')

In [19]:
t0 = time.time()
train(10)
print('{} seconds'.format(time.time() - t0))

0.0 seconds


In [20]:
Ys_test[0].unsqueeze(0)

tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.]], device='cuda:0')

In [21]:
forw = mod.forward(Xs_test)
guessed = torch.argmax(forw, axis=1).cpu().numpy()
print(sum(guessed == df_test["label"].values) / len(guessed))

0.901


In [22]:
# layers = linstorch.flattenseq(mod)
# layers

In [23]:
# back = linstorch.backwards(layers, 0, Ys_test[0].unsqueeze(0))

In [24]:
# import matplotlib.pyplot as plt

In [25]:
# plt.imshow(back.reshape(28,28).cpu().detach().numpy())

In [26]:
#del mod

In [27]:
#torch.cuda.empty_cache()

In [28]:
layers = linstorch.flattenseq(mod)
outpi = linstorch.forwardsto(layers, 100, Xs_test)
print(outpi.mean())

print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -1, Xs_test)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -2, Xs_test)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -3, Xs_test)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -4, Xs_test)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

tensor(0.1145, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9.8883, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.1000, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(9.8883, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-14.1018, device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.0065, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(3.4214, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.0006, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(3.4214, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-4.8169, device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.1325, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')


In [29]:
layers = linstorch.flattenseq(mod)
outpi = linstorch.forwardsto(layers, 100, Xs_train)
print(outpi.mean())

print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -1, Xs_train)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -2, Xs_train)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -3, Xs_train)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

print("===")
outpi = linstorch.forwardsto(layers, -4, Xs_train)
print(outpi.mean())
print(outpi.max())
print(outpi.min())

tensor(0.1139, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.5205, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.1000, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(1.5205, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.6109, device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.0064, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(11.1742, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0., device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.0006, device='cuda:0', grad_fn=<MeanBackward0>)
tensor(11.1742, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-17.8712, device='cuda:0', grad_fn=<MinBackward1>)
===
tensor(0.1307, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0., device='cuda:0')
